In [11]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import json
import ssl
import re
import time


In [12]:
def loadFiles():
    with open('restMasterList.json') as fp:
        f = json.load(fp)
        id = 20047
        print(len(f[str(id)]) > 0)
    return

In [13]:
def prepareMasterSeedList(district_elements, cuisine_elements, foodtype_elements):
    MasterSeedURL = []
    baseURL = "http://etw.nextdigital.com.hk/search/restaurant/?"

    
    for i in range(len(district_elements)):
        d = baseURL + "district=" + district_elements[i]
        MasterSeedURL.append(d)

    for i in range(len(cuisine_elements)):
        d = baseURL + "cuisine=" + cuisine_elements[i]
        MasterSeedURL.append(d)

    for i in range(len(foodtype_elements)):
        d = baseURL + "foodtype=" + foodtype_elements[i]
        MasterSeedURL.append(d)   

    print("Masterlist length: " +  str(len(MasterSeedURL)))
        
        
    return MasterSeedURL

In [14]:
def fetchSearchInfo(url):
    
    browser = webdriver.Chrome('/Users/samuelpun_old/Downloads/chromedriver')

    browser.get(url)
    time.sleep(1)

    elem = browser.find_element_by_tag_name("body")

        
    # extract Search Options seeds
    district_elements = extractElements(browser, "selected_district", "value", "find_elements_by_name")
    cuisine_elements = extractElements(browser, "selected_cuisine", "value", "find_elements_by_name")
    foodtype_elements = extractElements(browser, "selected_foodtype", "value", "find_elements_by_name")
    
    browser.close()
                                             
    return district_elements, cuisine_elements, foodtype_elements

In [15]:
def extractElements(browserObj, tagName, attr, method):
    
    if method == "find_elements_by_name":
        elements = browserObj.find_elements_by_name(tagName)
    elif method  == "find_elements_by_class_name":
        elements = browserObj.find_elements_by_class_name(tagName)
        
    snippets = []
    
    for item in elements:
        snippets.append(item.get_attribute(attr))
    
    return snippets

In [16]:
def crawlRestInfo(url, tag, pageDownNumber = 100, infScroll = False):
    
    if infScroll == False:
        ctx = ssl.create_default_context()
        ctx.check_hostname = False
        ctx.verify_mode = ssl.CERT_NONE
    
        html = urllib.request.urlopen(url, context=ctx).read()
        soup = BeautifulSoup(html, 'html.parser')
    
        # Retrieve all of the anchor tags
        snippets = soup(tag)
    
    else:
        browser = webdriver.Chrome('/Users/samuelpun_old/Downloads/chromedriver')

        browser.get(url)
        time.sleep(1)

        elem = browser.find_element_by_tag_name("body")

        # Generate auto pagedown
        lastHeight = browser.execute_script("return document.body.scrollHeight")
        while pageDownNumber:
            
            elem.send_keys(Keys.PAGE_DOWN)
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2.0)
            newHeight = browser.execute_script("return document.body.scrollHeight")
            if newHeight == lastHeight:
                break
            else:
                lastHeight = newHeight
            pageDownNumber-=1

        # extract restaurants class
        restclass_elements = extractElements(browser, "result-restaurant-list", 
                                             "innerHTML", "find_elements_by_class_name")
        browser.close()
        
                                             
    return restclass_elements

In [17]:
def processRestsData(restsEntry):
    
    count = 0
    
    # from a list of href exact restaurant and new masterList info
    for i in range(len(restsEntry)):
        #print(restsEntry[0])
        restID = re.findall('\\/restaurant\\/([0-9]+)\\/.*\\"',str(restsEntry[i]))
        #tags = re.findall('\\/restaurant\\/[0-9]+\\/(.*)\\"',str(restsEntry[i]))
        name = re.findall('ar-name\\"\\>(.+)\\<\\/div\\>',str(restsEntry[i]))
        tel = re.findall('ar-tel\\"\\>([0-9]+\s[0-9]+)\\<\\/div\\>',str(restsEntry[i]))
        addr = re.findall('ar-address\\"\\>(.+)\\<\\/div\\>',str(restsEntry[i]))
        tags = re.findall('ar-cuisine\\"\\>\\\n\s+(.+)\\<\\/div\\>',str(restsEntry[i]))
        
        rest = {}
        
        if len(restID) > 0:
            #print(rest)
            rest['ID'] = restID[0]
            
        if len(tags) > 0:
            tagsList = tags[0].split('|')
            for i in range(len(tagsList)):
                tagsList[i] = tagsList[i].strip()
            rest['tags'] = tagsList
            #print(rest['tags'])
            
        if len(name) > 0: 
            rest['name'] = name[0]
            
        if len(tel) > 0:     
            rest['tel'] = tel[0]
        
        if len(addr) > 0: 
            rest['addr'] = addr[0]
            
            if restID[0] in restMasterList:
                continue
            else:
                restMasterList[restID[0]] = rest
                count = count + 1
    
    print(str(len(restsEntry)) + " records found. " + str(count) + 
          " records updated. Total record: " + str(len(restMasterList)))

    # save file as JSON format
    with open('./restMasterList.json', 'w+') as fp:
        json.dump(restMasterList, fp)
        #existingRecord = dict(json.load(fp))
         
    return restMasterList

In [18]:
def processCrawlList(restMasterList, seedMasterList):
    
    pageDownNumber = 2000
    
    # scan thru the seedMasterList go get a list of href
    restsEntry = []
    count = 0
    
    for i in range(len(seedMasterList)):
        print("fetching " + str(i) + "th URL out of " + str(len(seedMasterList)))
        this_url = seedMasterList[i]
        restclass_elements = crawlRestInfo(this_url, 'a', pageDownNumber, True) 
        restMasterList = processRestsData(restclass_elements)
        #restsEntry =  restsEntry + restclass_elements
        count = count + 1
    
    print(str(count) + " url fetched")
    
    #restMasterList = processRestsData(restsEntry)
        
    return restsEntry, restMasterList

In [19]:
#url = input('Enter - ')
seedMasterList = []
restMasterList = {}
seedURL = "http://etw.nextdigital.com.hk"
url = 'http://etw.nextdigital.com.hk/search/restaurant/?cuisine=%E6%B8%AF%E5%BC%8F'
seedMasterList.append(url)

district_elements, cuisine_elements, foodtype_elements = fetchSearchInfo(seedURL)
seedMasterList = prepareMasterSeedList(district_elements, cuisine_elements, foodtype_elements)

restsEntry, restMasterList = processCrawlList(restMasterList, seedMasterList)

Masterlist length: 245
fetching 0th URL out of 245
0records found. 0 records updated. Total record: 0
fetching 1th URL out of 245
508records found. 508 records updated. Total record: 508
fetching 2th URL out of 245
895records found. 895 records updated. Total record: 1403
fetching 3th URL out of 245
380records found. 379 records updated. Total record: 1782
fetching 4th URL out of 245
159records found. 159 records updated. Total record: 1941
fetching 5th URL out of 245
20records found. 20 records updated. Total record: 1961
fetching 6th URL out of 245
57records found. 57 records updated. Total record: 2018
fetching 7th URL out of 245
31records found. 31 records updated. Total record: 2049
fetching 8th URL out of 245
42records found. 42 records updated. Total record: 2091
fetching 9th URL out of 245
97records found. 97 records updated. Total record: 2188
fetching 10th URL out of 245
15records found. 15 records updated. Total record: 2203
fetching 11th URL out of 245
55records found. 55 r

6records found. 1 records updated. Total record: 8032
fetching 97th URL out of 245
15records found. 3 records updated. Total record: 8035
fetching 98th URL out of 245
55records found. 11 records updated. Total record: 8046
fetching 99th URL out of 245
3records found. 0 records updated. Total record: 8046
fetching 100th URL out of 245
26records found. 3 records updated. Total record: 8049
fetching 101th URL out of 245
8records found. 2 records updated. Total record: 8051
fetching 102th URL out of 245
3records found. 0 records updated. Total record: 8051
fetching 103th URL out of 245
2records found. 1 records updated. Total record: 8052
fetching 104th URL out of 245
7records found. 0 records updated. Total record: 8052
fetching 105th URL out of 245
12records found. 1 records updated. Total record: 8053
fetching 106th URL out of 245
3records found. 1 records updated. Total record: 8054
fetching 107th URL out of 245
6records found. 0 records updated. Total record: 8054
fetching 108th URL o

fetching 193th URL out of 245
146records found. 1 records updated. Total record: 8635
fetching 194th URL out of 245
22records found. 0 records updated. Total record: 8635
fetching 195th URL out of 245
7records found. 0 records updated. Total record: 8635
fetching 196th URL out of 245
5records found. 0 records updated. Total record: 8635
fetching 197th URL out of 245
40records found. 0 records updated. Total record: 8635
fetching 198th URL out of 245
228records found. 11 records updated. Total record: 8646
fetching 199th URL out of 245
16records found. 2 records updated. Total record: 8648
fetching 200th URL out of 245
203records found. 12 records updated. Total record: 8660
fetching 201th URL out of 245
61records found. 0 records updated. Total record: 8660
fetching 202th URL out of 245
94records found. 3 records updated. Total record: 8663
fetching 203th URL out of 245
32records found. 2 records updated. Total record: 8665
fetching 204th URL out of 245
695records found. 0 records upda

In [26]:
with open('restMasterList.json') as fp:
    f = json.load(fp)
print(f)

{'41005': {'ID': '41005', 'tags': ['Café'], 'name': '13 Peel by Inferno Dynamics Coffee Roastery', 'tel': '9881 7013', 'addr': '中環卑利街13號地下'}, '40982': {'ID': '40982', 'tags': ['港式', '雲吞/餃子'], 'name': 'Home. Dumpling', 'tel': '9178 9269', 'addr': '中環利源東街25號地下'}, '40962': {'ID': '40962', 'tags': ['日本菜', '西餐廳'], 'name': 'The Balcony', 'tel': '2898 2184', 'addr': '中環皇后大道中118號3樓'}, '40931': {'ID': '40931', 'tags': ['印度菜', '酒吧'], 'name': 'Klassic', 'tel': '2554 7400', 'addr': '中環威靈頓街33號The Loop 6樓'}, '40900': {'ID': '40900', 'tags': ['西餐廳'], 'name': 'Café Crêpe', 'tel': '2907 1888', 'addr': '中環威靈頓街15B號地鋪'}, '40793': {'ID': '40793', 'tags': ['酒吧'], 'name': 'The Old Man', 'tel': '2703 1899', 'addr': '中環蘇豪鴨巴甸街37號'}, '25408': {'ID': '25408', 'tags': ['廣東菜', '點心', '中菜館'], 'name': '唐人館', 'tel': '2522 2148', 'addr': '中環皇后大道中15號置地廣場4樓411-413號'}, '25373': {'ID': '25373', 'tags': [''], 'name': '吉野家', 'tel': '2810 1125', 'addr': '中環港鐵香港站HOK21A號舖'}, '25365': {'ID': '25365', 'tags': [''], 'name': '吉野家', 